"""
This script is used to plot the Global mean surface air temperature (GMSAT) from observation and multimodel simulation.
"""

## import observation data

In [ ]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from pathlib import Path
import glob
# %%
# define function
import src.SAT_function_Obs_Fingerprint as data_process
import src.Data_Preprocess as preprocess

In [ ]:
# subplot a: global mean temperature anomalies during 1850-2022 HadCRUT5
input_observation = './Data/'

tas_HadCRUT_annual = xr.open_dataset(input_observation + 'tas_HadCRUT5_annual_anomalies.nc')

In [ ]:
tas_HadCRUT_annual

In [ ]:
# tas_HadCRUT_annual = tas_HadCRUT_annual.rename({'__xarray_dataarray_variable__': 'tas_HadCRUT5'})

In [ ]:
tas_HadCRUT_annual_1850_2022 = tas_HadCRUT_annual.sel(year=slice('1993', '2022')).tas

In [ ]:
tas_HadCRUT5_annual_ano = tas_HadCRUT_annual_1850_2022.mean(dim=['year'])

In [ ]:
tas_HadCRUT5_annual_ano.min().values, tas_HadCRUT5_annual_ano.max().values

## Input the multimodel mean GMST data

### Subplot b: plot the multimodel ensemble mean timeseries

In [ ]:
input_model = './Data/Timeseries/'

ACCESS_GMSAT_annual_ENS     = xr.open_dataset(input_model + 'GMSAT_ACCESS_annual_timeseries_ENS.nc')
CanESM_GMSAT_annual_ENS     = xr.open_dataset(input_model + 'GMSAT_CanESM5_annual_timeseries_ENS.nc')
EC_Earth_GMSAT_annual_ENS   = xr.open_dataset(input_model + 'GMSAT_EC_Earth_annual_timeseries_ENS.nc')
IPSL_GMSAT_annual_ENS   = xr.open_dataset(input_model + 'GMSAT_IPSL_CM6A_annual_timeseries_ENS.nc')
MIROC_GMSAT_annual_ENS  = xr.open_dataset(input_model + 'GMSAT_MIROC6_annual_timeseries_ENS.nc')
MPI_GMSAT_annual_ENS    = xr.open_dataset(input_model + 'GMSAT_MPI_ESM_annual_timeseries_ENS.nc')
MMEM_GMSAT_annual_ENS   = xr.open_dataset(input_model + 'GMSAT_SMILEs_ENS_annual_timeseries.nc')

In [ ]:
MMEM_GMSAT_annual_ENS_check = xr.open_dataset(input_model + 'GMSAT_SMILEs_ENS_annual_timeseries_obtained_basedOn_ModelENS.nc')

### Subplot c: plotting the alpha and beta coefficients spatial pattern

In [ ]:
# load the data
# MMEM GMSAT annual regression coefficient
dir_path = './Figure1/'

beta_MMEM_HadCRUT5_annual = xr.open_dataset(dir_path + 'HadCRUT_slope_Beta_coefficients_MMEM_TS.nc')
alpha_MMEM_HadCRUT5_annual = xr.open_dataset(dir_path + 'HadCRUT_intercept_Alpha_constant_MMEM_TS.nc')

In [ ]:
beta_MMEM_HadCRUT5_annual = beta_MMEM_HadCRUT5_annual.rename({'__xarray_dataarray_variable__': 'beta_MMEM_HadCRUT5'})
alpha_MMEM_HadCRUT5_annual = alpha_MMEM_HadCRUT5_annual.rename({'__xarray_dataarray_variable__': 'alpha_MMEM_HadCRUT5'})

In [ ]:
beta_MMEM_HadCRUT5_annual['beta_MMEM_HadCRUT5'].min().values, beta_MMEM_HadCRUT5_annual['beta_MMEM_HadCRUT5'].max().values

In [ ]:
alpha_MMEM_HadCRUT5_annual['alpha_MMEM_HadCRUT5'].min().values, alpha_MMEM_HadCRUT5_annual['alpha_MMEM_HadCRUT5'].max().values

In [ ]:
dir_MMEM = './Figure1/fingerprint_output/'
MMEM_beta_pattern = xr.open_dataset(dir_MMEM+'MMEM_GSAT_slope_Beta_coefficients_1850_2022.nc')

In [ ]:
MMEM_beta_pattern

### Subplot d: the reconstructed y mean and the y residual from the raw data

In [ ]:
dir_const = './Data/'

GSAT_estimate_forced = xr.open_dataset(dir_const + 'GSAT_HadCRUT5_Forced_anomalies_1850_2022.nc')
GSAT_estimate_internal = xr.open_dataset(dir_const + 'GSAT_HadCRUT5_Internal_Variability_anomalies_1850_2022.nc')

In [ ]:
GSAT_estimate_forced = GSAT_estimate_forced.rename({'__xarray_dataarray_variable__': 'GSAT_HadCRUT5_Forced'})
GSAT_estimate_internal = GSAT_estimate_internal.rename({'__xarray_dataarray_variable__': 'GSAT_HadCRUT5_Internal'})

In [ ]:
# Calculate the last 30years mean
GSAT_estimate_forced_1993_2022 = GSAT_estimate_forced.sel(year=slice('1993', '2022')).GSAT_HadCRUT5_Forced

In [ ]:
GSAT_estimate_forced_mean = GSAT_estimate_forced_1993_2022.mean(dim=['year'])
# GSAT_estimate_internal_mean = GSAT_estimate_internal['GSAT_HadCRUT5_Internal'].mean(dim=['year'])

In [ ]:
GSAT_estimate_forced_mean.min().values, GSAT_estimate_forced_mean.max().values

### Subplot e: the MMLE SAT anomalies during 1993-2022

In [ ]:
dir_MMLE = './time_dependency/Data/'
MMLE_ds = xr.open_dataset(dir_MMLE + 'SMILEs_annual_ano_1850_2022.nc')
MMLE_ds

In [ ]:
MMLE_30yr_mean = MMLE_ds.sel(year=slice('1993', '2022')).tas.mean(dim=['year'])

In [ ]:
MMLE_30yr_mean

## Plotting 

In [ ]:
import svgwrite
from matplotlib.lines import Line2D
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.legend import Legend
import matplotlib.lines as Line2D

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

In [ ]:
#Plotting
# setting the parameters for the figure
plt.rcParams['figure.figsize'] = (8, 10)
plt.rcParams['font.size'] = 16
# plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['ytick.major.right'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.bottom'] = True
# plt.rcParams['savefig.transparent'] = True # save the figure with a transparent background
plt.savefig("figure.eps", format="eps")


In [ ]:
# Setting the range of the x-axis
x = np.arange(1850, 2023, 1)

In [ ]:
obs_name = ["HadCRUT5"]
model_names = ["CanESM5(50)", "IPSL-CM6A-LR(32)", "EC-Earth3(21)", "ACCESS-ESM1.5(40)", "MPI-ESM1.2-LR(50)","MIROC6(50)"]

RGB_dict = {'CanESM5(50)':np.array([50, 34, 136])/255., 
            'IPSL-CM6A-LR(32)':np.array([68, 170, 152])/255., 
            'EC-Earth3(21)':np.array([221, 204, 118])/255., 
            'ACCESS-ESM1.5(40)':np.array([204, 101, 119])/255.,
            'MPI-ESM1.2-LR(50)':np.array([170, 67, 153])/255., 
            'MIROC6(50)':np.array([136, 33, 85])/255., 
            'MMLE':np.array([0, 0, 0])/255.}

In [ ]:
c_obs = "orange"
c_obs_1 = "blue"
c_obs_2 = "green" #'darkred'

obs_color = {
    "MLOST": c_obs,
    "HadCRUT5": c_obs_1,
    "Berkeley": c_obs_2
}

for obs in obs_name:
    print(obs)
    obs_color[obs]
    print(obs_color[obs])
    
lw_obs = 1.5
lw_model = 2.5
xmin, xmax = 1850, 2022
ymin, ymax = -1.5, 2.0

In [ ]:
RGB_dict[model_names[0]],model_names[0]

### Plot the y (original SAT anomalies data spatial pattern)

In [ ]:
# define the contourf plot function
def plot_data(data,lats,lons,levels=None, extend=None,cmap=None, title="", ax=None, show_xticks=False, show_yticks=False):
    if ax is None:
        fig, ax = plt.subplots(subplot_kw={'projection': ccrs.Robinson()})
    
    # ax.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=ccrs.PlateCarree())
    # Add coastlines
    ax.coastlines(resolution='110m')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,
                      colors='gray', alpha=0.15, linestyle='--', linewidth=0.25)

    # Disable labels on the top and right of the plot
    gl.top_labels = False
    gl.right_labels = False

    # Enable labels on the bottom and left of the plot
    gl.bottom_labels = show_xticks
    gl.left_labels = show_yticks
    gl.xformatter = cticker.LongitudeFormatter()
    gl.yformatter = cticker.LatitudeFormatter()
    gl.xlabel_style = {'size': 14}
    gl.ylabel_style = {'size': 14}
    
    if show_xticks:
        gl.bottom_labels = True
    if show_yticks:
        gl.left_labels = True
    # Add filled contour plot
    cf = ax.contourf(lons, lats, data, levels=levels, extend=extend, cmap=cmap, transform=ccrs.PlateCarree())
    
    # Add title
    ax.set_title(title, loc='center', fontsize=16, pad=5.0)
    return cf

In [ ]:
import matplotlib.pyplot as plt
import patchworklib as pw
import cartopy.util as cutil


In [ ]:
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colors import BoundaryNorm
import cartopy.util as cutil
import seaborn as sns
import matplotlib.colors as mcolors
import palettable
#  cmap = mcolors.ListedColormap(palettable.scientific.diverging.Vik_20.mpl_colors)
cmap=mcolors.ListedColormap(palettable.cmocean.diverging.Balance_20.mpl_colors)

In [ ]:
# Four subplots
"""
    subplot a: global mean temperature anomalies during 1850-2022 HadCRUT5; contourf plot
    subplot b: model simulated GSAT timeseries in each SMILEs and MMEM 1850-2022; x-y line plot
    subplot c: GSAT anomalies Beta and Alpha coefficients; contourf plot (from top to bottom: Beta, Alpha two subplots concatenated)
    subplot d: GSAT anomalies spatial pattern by construction, with the internal variability and forced component; contourf plot 
    (from top to bottom: forced component, internal variability two subplots concatenated)
"""

In [ ]:
# subplot b: model simulated GSAT timeseries in each SMILEs and MMEM 1850-2022
# plot the GMSAT timeseries
fig2, ax2 = plt.subplots(figsize=(15, 6))
plt.plot([xmin,xmax],[0,0], color='grey', linestyle='-', linewidth=0.75)
# seven lines for six SMILEs and one MMEM
ax2.plot(x, CanESM_GMSAT_annual_ENS['tas'], color=RGB_dict[model_names[0]], label=model_names[0], linewidth=lw_model,
         linestyle= '--', alpha=0.75)
ax2.plot(x, IPSL_GMSAT_annual_ENS['tas'], color=RGB_dict[model_names[1]], label=model_names[1], linewidth=lw_model,
            linestyle= '--', alpha=0.75)
ax2.plot(x, EC_Earth_GMSAT_annual_ENS['tas'], color=RGB_dict[model_names[2]], label=model_names[2], linewidth=lw_model,
            linestyle= '--', alpha=0.75)
ax2.plot(x, ACCESS_GMSAT_annual_ENS['tas'], color=RGB_dict[model_names[3]], label=model_names[3], linewidth=lw_model,
            linestyle= '--', alpha=0.75)
ax2.plot(x, MPI_GMSAT_annual_ENS['tas'], color=RGB_dict[model_names[4]], label=model_names[4], linewidth=lw_model,
            linestyle= '--', alpha=0.75)
ax2.plot(x, MIROC_GMSAT_annual_ENS['tas'], color=RGB_dict[model_names[5]], label=model_names[5], linewidth=lw_model,
            linestyle= '--', alpha=0.75)
ax2.plot(x, MMEM_GMSAT_annual_ENS['tas'], color=RGB_dict['MMLE'], label='MMLE(243)', linewidth=lw_model)

# Add legend
plt.axvspan(1950, 2022, alpha=0.25, color='grey')
plt.text(1950, 2.05, '1950-2022', fontsize=18)
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylabel('SAT anomaly relative to 1961-1990(°C)', fontsize=18)
legend = plt.legend(loc='upper left', fontsize=14, ncol=2, title='CMIP6 Models', title_fontsize='16')
plt.text(1843, 2.15, 'a', fontsize=22, fontweight='bold', ha='left')
plt.text(1849, 2.15, r'$\langle \bar{x}_t \rangle$'+' timeseries', fontsize=20)
plt.savefig("Fig-1a.png", dpi=300, bbox_inches='tight')
plt.savefig("Fig-1a.pdf", dpi=300, bbox_inches='tight')
plt.savefig("Fig-1a.eps", format='eps', dpi=300, bbox_inches='tight')

plt.show()

In [ ]:
input_diff_dir =  './Revised_main_figures/Figure1_Method_schematic/data/'

MMEM_beta_pattern_diff = xr.open_dataset(input_diff_dir + 'MMEM_GSAT_slope_Beta_coefficients_diff_1850_2022.nc')
Forced_anomalies_diff = xr.open_dataset(input_diff_dir + 'MMEM_GSAT_Forced_anomalies_diff_1993_2022.nc')

In [ ]:
MMEM_beta_pattern_diff

In [ ]:
GSAT_estimate_forced_mean.min().values, GSAT_estimate_forced_mean.max().values

In [ ]:
Forced_anomalies_diff

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.util as cutil
from matplotlib.gridspec import GridSpec

# contour settings
bounds_fp   = np.arange(-1.0, 1.1, 0.1)
cmap_fp     = 'RdBu_r'
bounds_anom = np.arange(-1.0, 1.1, 0.1)
cmap_anom   = 'RdBu_r'

fig = plt.figure(figsize=(18, 10))
gs  = GridSpec(2, 3, figure=fig, wspace=0.05, hspace=0.05)

# ────────────────────────────────────────────────────────────────────────────
# Top row: fingerprint patterns (b,c,d)
# ────────────────────────────────────────────────────────────────────────────
titles_fp = [rf"OBS ($\beta$)", rf"MMLE ($\beta$)", "c - b"]
letters   = ['b','c','d']
axes_fp   = []   # collect them here

for i, da in enumerate([beta_MMEM_HadCRUT5_annual['beta_MMEM_HadCRUT5'], MMEM_beta_pattern['coefficients'], MMEM_beta_pattern_diff["slope_diff"]]):
    ax = fig.add_subplot(gs[0, i], projection=ccrs.Robinson(180))
    data_cyc, lon_cyc = cutil.add_cyclic_point(da, coord=da.lon)
    im_fp = plot_data(
        data_cyc, da.lat, lon_cyc,
        levels=bounds_fp, cmap=cmap_fp, extend='both',
        title=titles_fp[i], ax=ax,
        show_xticks=True, show_yticks=(i==0)
    )
    ax.text(-0.05, 1.05, letters[i],
            transform=ax.transAxes,
            fontsize=20, fontweight='bold')
    axes_fp.append(ax)
# ────────────────────────────────────────────────────────────────────────────
# Bottom row: composite anomalies (e,f,g)
# ────────────────────────────────────────────────────────────────────────────
titles_an  = ['OBS ' + '(' +r'$\hat{y} = \beta_{i,j} \langle \bar{x}_t \rangle + \alpha_{i,j}$'+ ')', 'MMLE ' + '('+ r'$y_{MMLE}$'+')', "f - e"]
letters_an = ['e','f','g']
axes_an    = []  # collect these

for i, da in enumerate([GSAT_estimate_forced_mean, MMLE_30yr_mean, Forced_anomalies_diff["Forced_anomalies_diff"]]):
    ax = fig.add_subplot(gs[1, i], projection=ccrs.Robinson(180))
    data_cyc, lon_cyc = cutil.add_cyclic_point(da, coord=da.lon)
    im_an = plot_data(
        data_cyc, da.lat, lon_cyc,
        levels=bounds_anom, cmap=cmap_anom, extend='both',
        title=titles_an[i], ax=ax,
        show_xticks=True, show_yticks=(i==0)
    )
    ax.text(-0.05, 1.05, letters_an[i],
            transform=ax.transAxes,
            fontsize=20, fontweight='bold')
    axes_an.append(ax)

# create a narrow axes at the bottom of row 0
cax_fp = fig.add_axes([0.3, 0.52, 0.45, 0.025])  
cb_fp  = fig.colorbar(
    im_fp,           # last mappable from the fp‐loop
    cax=cax_fp,      # place it in that new axes
    orientation='horizontal'
)
cb_fp.set_label('Regression coefficient', fontsize=16)
cb_fp.ax.tick_params(labelsize=14)
cb_fp.ax.minorticks_off()


# ────────────────────────────────────────────────────────────────────────────
# shared horizontal colorbar for the anomalies row
# ────────────────────────────────────────────────────────────────────────────
# create a narrow axes at the bottom of the figure
cax_an = fig.add_axes([0.3, 0.12, 0.45, 0.025])
cb_an  = fig.colorbar(
    im_an,           # last mappable from the anom‐loop
    cax=cax_an,      # place it in that new axes
    orientation='horizontal'
)
cb_an.set_label('SAT anomaly (°C)', fontsize=16)
cb_an.ax.tick_params(labelsize=14)
cb_an.ax.minorticks_off()
# ────────────────────────────────────────────────────────────────────────────
# Save & show
# ────────────────────────────────────────────────────────────────────────────
os.makedirs("figures", exist_ok=True)
for ext in ("png", "pdf", "eps"):
    fig.savefig(f"figures/Fig1_b-g_horizontal_cbars.{ext}",
                dpi=300, bbox_inches="tight")
plt.show()
